### https://www.kaggle.com/code/deepak180222/question-answering-using-bloom-7b1-and-chromadb

In [1]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-acef07c8-f672-cc03-6135-501783b9f605)


In [2]:
%%time

! pip install -qq -U langchain tiktoken pypdf chromadb faiss-gpu
! pip install -qq -U transformers InstructorEmbedding sentence_transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-pubsublite 1.8.2 requires overrides<7.0.0,>=6.0.1, but you have overrides 7.4.0 which is incompatible.
jupyterlab-lsp 4.2.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
CPU times: user 518 ms, sys: 123 ms, total: 641 ms
Wall time: 50.6 s


In [3]:
import os
import textwrap
import glob
import shutil

import langchain
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA, VectorDBQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

/opt/conda/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
#os.makedirs('/kaggle/working/mk-vectordb-chroma')

In [5]:
#shutil.rmtree('/kaggle/working/mk-vectordb-chroma',ignore_errors=True)

In [6]:
#shutil.copytree('/kaggle/input/mkvc-2-part-1/kaggle/working/mk-vectordb-chroma-two', '/kaggle/working/mk-vectordb-chroma-two')

In [7]:
#shutil.rmtree('/kaggle/working/dataset',ignore_errors=True)

In [8]:
#os.makedirs('/kaggle/working/dataset')

In [9]:
#os.remove('/kaggle/working/file.zip')

In [10]:
%%time

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="intfloat/multilingual-e5-small", model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: user 6.66 s, sys: 4.34 s, total: 11 s
Wall time: 1min 7s


In [11]:
noke = 0
based_ke_from = 0
actual_from = 0
actual_to = 0

for x in range(40):
    noke += 1
    
    actual_from = based_ke_from+1
    actual_to = based_ke_from+100 
    
    print(noke,actual_from," ~ ",actual_to)
    based_ke_from = actual_to
    
    shutil.rmtree('/kaggle/working/dataset',ignore_errors=True)
    os.makedirs('/kaggle/working/dataset')
    
    #Pindahkan 100
    files = glob.glob("/kaggle/input/mkri-fullset/*")
    ke = 0
    for file in files:
        ke += 1
        if ke >= actual_from and ke <= actual_to:
            shutil.copy(file, '/kaggle/working/dataset/')
            
    print(len(glob.glob('/kaggle/working/dataset/*')))
    
    loader = DirectoryLoader('/kaggle/working/dataset/', 
                             glob="./*.pdf",
                             loader_cls=PyPDFLoader,
                             show_progress=True,
                             use_multithreading=True)

    documents = loader.load()
    
    ### clean a bit
    for i in range(len(documents)):
        documents[i].page_content = documents[i].page_content.replace('\t', ' ')\
                                                             .replace('\n', ' ')\
                                                             .replace('       ', ' ')\
                                                             .replace('      ', ' ')\
                                                             .replace('     ', ' ')\
                                                             .replace('    ', ' ')\
                                                             .replace('   ', ' ')\
                                                             .replace('  ', ' ')
        
    print(len(documents))
    
    #print(documents[30].page_content)
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    print(len(texts))
    
    persist_directory = 'mk-vectordb-chroma-eight'

    ### create embeddings and DB
    vectordb = Chroma.from_documents(documents=texts,
                                     embedding=instructor_embeddings,
                                     persist_directory=persist_directory,
                                     collection_name='mk_docs')


    ### persist Chroma database
    vectordb.persist()

1 1  ~  100
100


100%|██████████| 100/100 [03:52<00:00,  2.32s/it]


6761
16053
2 101  ~  200
100


100%|██████████| 100/100 [05:34<00:00,  3.34s/it]


8732
20912
3 201  ~  300
100


...

8764
21540
39 3801  ~  3900
100


100%|██████████| 100/100 [03:52<00:00,  2.32s/it]


7445
17887
40 3901  ~  4000
94


100%|██████████| 94/94 [03:53<00:00,  2.48s/it]


6862
17065


In [12]:
%%time

!zip -r file.zip /kaggle/working/mk-vectordb-chroma-eight

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: kaggle/working/mk-vectordb-chroma-eight/ (stored 0%)
  adding: kaggle/working/mk-vectordb-chroma-eight/chroma.sqlite3 (deflated 56%)
  adding: kaggle/working/mk-vectordb-chroma-eight/5d71f268-ccd5-4b21-8d07-d7e0384ae0b6/ (stored 0%)
  adding: kaggle/working/mk-vectordb-chroma-eight/5d71f268-ccd5-4b21-8d07-d7e0384ae0b6/header.bin (deflated 52%)
  adding: kaggle/working/mk-vectordb-chroma-eight/5d71f268-ccd5-4b21-8d07-d7e0384ae0b6/data_level0.bin (deflated 12%)
  adding: kaggle/working/mk-vectordb-chroma-eight/5d71f268-ccd5-4b21-8d07-d7e0384ae0b6/link_lists.bin (deflated 68%)
  adding: kaggle/working/mk-vectordb-chroma-eight/5d71f268-ccd5-4b21-8d07-d7e0384ae0b

In [13]:
#shutil.rmtree('/kaggle/working/dataset',ignore_errors=True)
#os.remove('/kaggle/working/file.zip')